# 3. Train_CNN

**Tensorboard**
- Input at command: tensorboard --logdir=./log
- Input at browser: http://127.0.0.1:6006

In [1]:
import time
import os
import pandas as pd

project_name = 'SceneClassification'
step_name = 'Train_CNN'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

cwd = os.getcwd()
model_path = os.path.join(cwd, 'model')
log_path = os.path.join(cwd, 'log')
print('model_path: ' + model_path)
print('model_path: ' + log_path)

run_name: SceneClassification_Train_CNN_20171028_150251
model_path: E:\SceneClassification\model
model_path: E:\SceneClassification\log


In [2]:
test_images = os.listdir(os.path.join(cwd, 'input', 'data_test_a', 'test'))
print(test_images[0:10])

['00002ff812f48a3df27c321d517a6300ed8da0c3.jpg', '00049a860dca2af378faeb0ee6f435c6063818ef.jpg', '0011a9c9216c3763ffc33641a8ffc975127dc404.jpg', '0045a44cacc7bc9826db9b54d2dcd70b810250f9.jpg', '004b6823145471c6a4ce292e864909fde2d04969.jpg', '0056e4d54eee781117c9d407d03ebf7192126b1f.jpg', '005763f88b25b18ae524b25afcce960403665383.jpg', '005b5444df96e3a155f2a73a8dccc0267e118413.jpg', '005c6ba205a246d0d3c8f73adfd4398b8e483962.jpg', '005de85662d754f98a1476a37b189902800ace91.jpg']


In [3]:
%%time
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

x_train = []
y_train = {}
x_val = []
y_val = {}
x_test = []

cwd = os.getcwd()
feature_cgg16 = os.path.join(cwd, 'model', 'feature_VGG16_{}.h5'.format(171023))
feature_cgg19 = os.path.join(cwd, 'model', 'feature_VGG19_{}.h5'.format(171023))
feature_resnet50 = os.path.join(cwd, 'model', 'feature_ResNet50_{}.h5'.format(171023))
feature_mobilenet = os.path.join(cwd, 'model', 'feature_MobileNet_{}.h5'.format(171023))
feature_xception = os.path.join(cwd, 'model', 'feature_Xception_{}.h5'.format(171023))
feature_inception = os.path.join(cwd, 'model', 'feature_InceptionV3_{}.h5'.format(171023))
for filename in [feature_cgg16, feature_cgg19, feature_resnet50, feature_mobilenet, feature_xception, feature_inception]:
    with h5py.File(filename, 'r') as h:
        x_train.append(np.array(h['train']))
        y_train = np.array(h['train_label'])
        x_val.append(np.array(h['val']))
        y_val = np.array(h['val_label'])
        x_test.append(np.array(h['test']))

# print(x_train[0].shape)
x_train = np.concatenate(x_train, axis=-1)
# y_train = np.concatenate(y_train, axis=0)
x_val = np.concatenate(x_val, axis=-1)
# y_val = np.concatenate(y_val, axis=0)
x_test = np.concatenate(x_test, axis=-1)
print(x_train.shape)
print(x_train.shape[1:])

print(len(y_train))
print(x_val.shape)
print(len(y_val))
print(x_test.shape)

(53879, 8192)
(8192,)
53879
(7120, 8192)
7120
(7040, 8192)
Wall time: 6.2 s


In [4]:
# x_train, y_train = shuffle(x_train, y_train)
print(x_train.shape)
print(y_train.shape)

(53879, 8192)
(53879,)


In [5]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
print(y_train.shape)
print(y_val.shape)

Using TensorFlow backend.


(53879, 80)
(7120, 80)


### Reshape array to image

In [6]:
target_shape = (91, 91)

extend_widgth = target_shape[0]*target_shape[1] - x_train.shape[1]
print('Target shape: ', target_shape)
print('Extend_widgth: ', extend_widgth)
print('Before extend:')
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

x_train_ext = np.zeros((x_train.shape[0], extend_widgth))
x_val_ext = np.zeros((x_val.shape[0], extend_widgth))
x_test_ext = np.zeros((x_test.shape[0], extend_widgth))

x_train0 = np.column_stack((x_train, x_train_ext))
x_val0 = np.column_stack((x_val, x_val_ext))
x_test0 = np.column_stack((x_test, x_test_ext))

print('After extend:')
print(x_train0.shape)
print(x_val0.shape)
print(x_test0.shape)

x_train0 = x_train0.reshape(-1, target_shape[0], target_shape[1], 1)
x_val0 = x_val0.reshape(-1, target_shape[0], target_shape[1], 1)
x_test0 = x_test0.reshape(-1, target_shape[0], target_shape[1], 1)

print('After reshape:')
print(x_train0.shape)
print(x_val0.shape)
print(x_test0.shape)

Target shape:  (91, 91)
Extend_widgth:  89
Before extend:
(53879, 8192)
(7120, 8192)
(7040, 8192)
After extend:
(53879, 8281)
(7120, 8281)
(7040, 8281)
After reshape:
(53879, 91, 91, 1)
(7120, 91, 91, 1)
(7040, 91, 91, 1)


### Build CNN

In [7]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard

In [8]:
def ge_lr(x):
    lr = 1.5e-3 * 0.9 ** x
    if lr < 1e-4:
        lr = 1e-4
    print(lr, end=' ')
    return lr
        
# annealer = LearningRateScheduler(lambda x: 1.1e-4 * 0.9 ** x)
annealer = LearningRateScheduler(ge_lr)

log_dir = os.path.join(log_path, run_name)
print('log_dir:' + log_dir)
tensorBoard = TensorBoard(log_dir=log_dir)

log_dir:E:\SceneClassification\log\SceneClassification_Train_CNN_20171028_150251


In [9]:
%%time

model = Sequential()
# Block 1
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu', padding = 'Same',
                 input_shape = (91, 91, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu', padding = 'Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=(2,2)))
model.add(Dropout(0.5))
# Block 2
# model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation='relu', padding = 'Same'))
# model.add(BatchNormalization())
# model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation='relu', padding = 'Same'))
# model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation='relu', padding = 'Same'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation='relu', padding = 'Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=(2,2)))
model.add(Dropout(0.5))
# Block 3
# model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation='relu', padding = 'Same'))
# model.add(BatchNormalization())
# model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation='relu', padding = 'Same'))
# model.add(BatchNormalization())
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation='relu', padding = 'Same'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation='relu', padding = 'Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=(2,2)))
model.add(Dropout(0.5))
# Block 4
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation='relu', padding = 'Same'))
# model.add(BatchNormalization())
# model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation='relu', padding = 'Same'))
# model.add(BatchNormalization())
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation='relu', padding = 'Same'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation='relu', padding = 'Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=(2,2)))
model.add(Dropout(0.5))
# Block 5
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation='relu', padding = 'Same'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 512, kernel_size = (3, 3), activation='relu', padding = 'Same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=(2,2)))
model.add(Dropout(0.25))

# Output
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(80, activation = 'softmax'))

model.compile(optimizer = Adam(lr=1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

hist = model.fit(x_train0, y_train, 
                 batch_size = 256, 
                 verbose=1,
                 epochs = 100,
                 validation_data=(x_val0, y_val), 
                 callbacks=[annealer, tensorBoard])

Train on 53879 samples, validate on 7120 samples
0.0015 Epoch 1/100
53879/53879 [==============================] - 215s - loss: 4.3613 - acc: 0.0350 - val_loss: 4.8793 - val_acc: 0.0108
0.00135 Epoch 2/100
53879/53879 [==============================] - 210s - loss: 3.8653 - acc: 0.0753 - val_loss: 4.5622 - val_acc: 0.0397
0.0012150000000000002 Epoch 3/100
53879/53879 [==============================] - 210s - loss: 3.3620 - acc: 0.1465 - val_loss: 3.2656 - val_acc: 0.1525
0.0010935 Epoch 4/100
53879/53879 [==============================] - 210s - loss: 2.9534 - acc: 0.2248 - val_loss: 2.7623 - val_acc: 0.2712
0.00098415 Epoch 5/100
53879/53879 [==============================] - 210s - loss: 2.5844 - acc: 0.3113 - val_loss: 2.3625 - val_acc: 0.3619
0.0008857350000000001 Epoch 6/100
53879/53879 [==============================] - 211s - loss: 2.2791 - acc: 0.3889 - val_loss: 1.8936 - val_acc: 0.4742
0.0007971615000000001 Epoch 7/100
53879/53879 [==============================] - 212s - los

KeyboardInterrupt: 

In [ ]:
final_loss, final_acc = model.evaluate(x_val0, y_val, verbose=0)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

Final loss: 1.2017, final accuracy: 0.7065


In [ ]:
val_preds = model.predict(x_val0)
print(val_preds.shape)
print(y_val.shape)

In [ ]:
from sklearn.metrics import log_loss, accuracy_score

print('Val log_loss: {}'.format(log_loss(y_val, val_preds)))
val_proba_limited = np.clip(val_preds, 0.005, 0.995)
print('Val limited log_loss: {}'.format(log_loss(y_val, val_proba_limited)))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

## Output

In [ ]:
run_name0 = run_name + '_' + str(int(final_acc*10000)).zfill(4)
print('run_name: ' + run_name0)

In [ ]:
def saveModel(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigthsFile = os.path.join(modelPath, run_name + '.h5')
    model.save(weigthsFile)
saveModel(model, run_name0)

In [ ]:
print('Done !')